In [56]:
import vk_api
import time
import requests
import json
import os

from vk_api.longpoll import VkLongPoll, VkEventType
from bs4 import BeautifulSoup as bs

In [69]:
class User:
    def __init__(self, user_id, group, name):
        self.user_id = user_id
        self.group = group
        self.name = name
    
    @staticmethod
    def getById(user_id):
        user_file = open(f"user{user_id}.txt")
        user_prop = eval(user_file.read())
        user_file.close()
        
        user = User(user_prop["user_id"], user_prop["name"], user_prop["group"])
        
        return user
    
    @staticmethod
    def is_exist(user_id):
        return os.path.isfile(f"user{user_id}.txt")
    
    def get_assistant(self):
        html_doc = requests.get(f"http://www.ifmo.ru/ru/schedule/0/{self.group}/0/raspisanie_zanyatiy_{self.group}.htm")
        
        return html_doc
    
    def save(self):
        user_file = open(f"user{self.user_id}.txt", "w")
        
        user_file.write(str({
            "user_id": self.user_id,
            "group": self.group.upper(),
            "name": self.name
        }))
        
        user_file.close()

In [58]:
user = User.getById(313489913)

In [59]:
doc = user.get_assistant()

In [18]:
doc.html

AttributeError: 'Response' object has no attribute 'html'

In [37]:
soup = bs(doc.text, 'html.parser')



<span>Пн</span>

In [53]:
table1 = soup.find(id="1day")
times_list = table1.find_all("td", attrs={"class": "time"})

In [54]:
times_list = [time.span.text for time in times_list]

display(times_list)

['08:20-09:50']

In [71]:
class Bot:
    def __init__(self, vk_token):
        self.vk_token = vk_token
        self.vk = vk_api.VkApi(token = vk_token)
        self.longpoll = VkLongPoll(self.vk)
        self._COMMANDS = ["/start", "/weather", "/help"]
        
    def write_msg(self, user_id, message):
        self.vk.method('messages.send', {'user_id': user_id, 'message': message, 'random_id': 0})
        
    def answer(self, request, user_id):
        message = "Я вас не понимаю(\nВведите /help чтобы получить справку обо мне"
        
        if len(request.split(",")) == 2:
            user = User(
                user_id,
                request.split(",")[1],
                request.split(",")[0],
            )
        
            user.save()

            message = f"Я вас запомнил, {user.name}! Постараюсь каждое утро вас веселить и держать в курсе дел!"
        
        elif request == "/start":
            if User.is_exist(user_id):
                message = f"Я то вас уже знаю, {User.getById(user_id).name}"
            else:
                message = "Отлично, пожалуйста, введите через запятую то, как я могу вас называть и вашу группу)"
        
        elif request == "/help":
            message = (
                       "/start - регистрация\n" + 
                       "/weather - получить информацию о погоде\n" + 
                       "/help - получить справку\n"
            )
        elif request == "/weather":
            message = get_weather_data()
        elif request == "/assistant":
            message = get_assistant(user_id, 0)
            
        self.write_msg(user_id, message)
        
    def run(self):
        for event in self.longpoll.listen():   
            if event.type == VkEventType.MESSAGE_NEW:
                if event.to_me:
                    request = event.text

                    self.answer(request, event.user_id)
                
    

In [61]:
#global constants
weather_api_key = "af934391a420f5f65302ab3d0691e315"
vk_token = "53a987ba81983d78efb3f3cc228d8b172146256cd8e65fb900064f6d216f5f6a999ec7d0d9e1cc11486ff"

In [62]:
def get_time():
    t = time.localtime()
    current_time = time.strftime("%H:%M:%S", t)
    return current_time

In [63]:
def get_weather_data():
    global weather_api_key

    weather_data = requests.get(
        "https://api.openweathermap.org/data/2.5/weather?" +
            "q=saint petersburg&" + 
            "units=metric&" +
            "appid=" + weather_api_key
    )
    
    message = get_recomendationbyDegree(weather_data.json()["main"]["temp_min"])
    
    return message

In [64]:
def get_recomendationbyDegree(degree):
    text = "Сегодня минимальная температура составляет " + str(degree) + " градусов по шкале Цельсия. "
    
    if degree > 10:
        text += ("Значит можно не надевать подштанники, но осторожно)  \n " + 
                "Не забудьте только шарфик и шапку с собой взять)")
    elif degree > 0:
        text += "НАДЕВАЙТЕ ПОДШТАННИКИ!!!!"
    elif degree > -10:
        text += "Если у вас ничего нет под штанами, то RIP AND PEACE"
    elif degree > -20:
        text += "Сегодня очень холодно! Брр"
    elif degree > -30:
        text += "Дети из Якутии терпят, и вы терпите)"
    elif degree > -40:
        text += "Это уже не шутки."
    else:
        text += "НУУУ ООчень холодно(("
    return text

In [ ]:
bot = Bot(vk_token)
bot.run()